In [9]:
import pandas as pd
import numpy as np
import spacy
import json
from spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load("en_core_web_sm")
from IPython.display import clear_output
clear_output()

In [10]:
DATASET_FILE =r"C:\Users\prabigya\Desktop\work_here\Recommendo-Personalized-Shopping-Powered-by-LLMs\recommedation_algo_system\fine_tuning\dataset\flipkart_com-ecommerce_sample.csv"
df = pd.read_csv(DATASET_FILE)
df = df.drop_duplicates(subset = ["pid"])
df = df[['product_name','product_category_tree','description','brand','discounted_price','product_specifications',"overall_rating"]]
df2 = df['product_name']
df = df.astype(str)
df = df.dropna()
df = df.applymap(lambda x: x.lower())

C:\Users\prabigya\AppData\Local\Temp\ipykernel_5184\4135301190.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.lower())


In [11]:

import re
# Function to extract keys and values from product_specifications column
def extract_specifications(specifications):
    pairs = re.findall(r'"key"=>"(.*?)", "value"=>"(.*?)"', specifications)
    pairs_formatted = [f"Feature  {key}:{value}." for key, value in pairs]
    return ' '.join(pairs_formatted)

def apply_remove_gret_sign(text):
    text=str(text).replace("[","").replace("]","").replace(">>",",").replace(" ,",",")
    return text
def capitalize_each_letter_text(text):
    text=str(text).title()
    return text

df['product_specifications'] = df['product_specifications'].apply(extract_specifications)
df["product_category_tree"]=df["product_category_tree"].apply(apply_remove_gret_sign)
df["product_name"]=df["product_name"].apply(capitalize_each_letter_text)

df_no_rating = df[df["overall_rating"] == "no rating available"]
df_with_rating = df[df["overall_rating"] != "no rating available"]
sampled_no_rating = df_no_rating.sample(n=len(df_with_rating) // 10, random_state=1)

# Combine the two DataFrames
rdf = pd.concat([df_with_rating, sampled_no_rating]).sample(frac=1, random_state=1).reset_index(drop=True)



filtered_rdf = rdf[
    (rdf['product_category_tree'].notnull()) & (rdf['description'].notnull()) &
    (rdf['product_category_tree'].str.len() > 25) & (rdf['description'].str.len() > 25)
]
filtered_rdf["description"]=filtered_rdf["description"]+" Its rating is "+filtered_rdf["overall_rating"]+"."
necessary_cols=filtered_rdf
necessary_cols.to_csv(r"C:\Users\prabigya\Desktop\work_here\Recommendo-Personalized-Shopping-Powered-by-LLMs\recommendation_system_ecommerce\dataset\usable_data.csv",index=False)
necessary_data_dict = necessary_cols.to_dict(orient='index')


output_file_path = r'C:\Users\prabigya\Desktop\work_here\Recommendo-Personalized-Shopping-Powered-by-LLMs\recommendation_system_ecommerce\dataset\vector_storing_documents.json'
with open(output_file_path, 'w') as json_file:
    json.dump(necessary_data_dict, json_file, indent=4)

C:\Users\prabigya\AppData\Local\Temp\ipykernel_5184\3197736538.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rdf["description"]=filtered_rdf["description"]+" Its rating is "+filtered_rdf["overall_rating"]+"."


In [12]:
import os
import json
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.documents import Document
from sentence_transformers import SentenceTransformer


c:\Users\prabigya\Desktop\work_here\Recommendo-Personalized-Shopping-Powered-by-LLMs\recommend_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
from langchain.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pydantic import BaseModel, ConfigDict, Field, SecretStr
from langchain_core.embeddings import Embeddings
from typing import Any, Dict, List, Optional

class FineTunedHuggingFaceModel(Embeddings):
    def __init__(self, model_name: str):
        """Initialize with the model name or path."""
        self.model_name = model_name
        self.model = SentenceTransformer(model_name, device="cpu")

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        # Get the embeddings for the documents
        embeddings = self.model.encode(texts, convert_to_tensor=True)

        # Convert the tensor embeddings into a list of lists (each inner list represents an embedding)
        return embeddings.cpu().numpy().tolist()

    def embed_query(self, text: str) -> List[float]:
        return self.embed_documents([text])[0]


In [14]:
model_dir=r"C:\Users\prabigya\Desktop\work_here\Recommendo-Personalized-Shopping-Powered-by-LLMs\bge_models_weights"
fine_tuned_model = SentenceTransformer(
    model_dir, device="cpu")
embeddings=FineTunedHuggingFaceModel(model_name=model_dir)

In [15]:
print(f"Embedding dimension: {len(embeddings.embed_query('example query'))}")


Embedding dimension: 768


In [16]:

def create_vector_store(input_file_folder_loc,output_file_folder_loc,embedding_model):
    os.makedirs(output_file_folder_loc, exist_ok=True)
    files_in_output_folder=os.listdir(output_file_folder_loc)
    if len(files_in_output_folder)>0:
        return 
    document_created_list=[]
    for file_name in os.listdir(input_file_folder_loc):
        input_file_path = os.path.join(input_file_folder_loc, file_name)

        if os.path.isfile(input_file_path) and file_name.lower().endswith(".json"):
            try:
                loaded_data=json.load(open(input_file_path))

                for key,value in loaded_data.items():
                    necessary_data = {}
                    try:
                        
                        necessary_data["product_name"]=value["product_name"]
                        necessary_data["overall_rating"]=value["overall_rating"]
                        necessary_data["product_category_tree"]=value["product_category_tree"]
                        necessary_data["brand"]=value["brand"]
                        product_specification=value["product_specifications"]
                        document = Document(
                            page_content=product_specification,
                            metadata=necessary_data
                        )
                        document_created_list.append(document)
                    except:
                        pass
            
            except Exception as e:
                print(f"Error processing {file_name}: {e}")
    
    vector_store = FAISS.from_documents(document_created_list, embedding_model)
    vector_store.save_local(output_file_folder_loc)


training_data_path=r"C:\Users\prabigya\Desktop\work_here\Recommendo-Personalized-Shopping-Powered-by-LLMs\recommendation_system_ecommerce\dataset"
output_vector_loc=r'C:\Users\prabigya\Desktop\work_here\Recommendo-Personalized-Shopping-Powered-by-LLMs\recommendation_system_ecommerce\dataset\vector_store'
create_vector_store(training_data_path,output_vector_loc,embeddings)


In [17]:
output_vector_loc=r'C:\Users\prabigya\Desktop\work_here\Recommendo-Personalized-Shopping-Powered-by-LLMs\recommendation_system_ecommerce\dataset\vector_store'
loaded_vector_store = FAISS.load_local(output_vector_loc, embeddings,allow_dangerous_deserialization=True)

In [34]:
# Perform a similarity search
query = "mobile"
results = loaded_vector_store.similarity_search_with_score(query,k=4)

# Check similarity scores and proceed if greater than threshold
for result, score in results:
    if len(str(result.page_content))>20:
        print(f"Content: {result.page_content}")
        print(f"Metadata: {result.metadata}")
        print(f"Dissimilarity score: {score}")
        print("\n","-"*50)

Content: Feature  in the box:mobile holder. Feature  brand:bracketron. Feature  model:tour window hip kicker. Feature  color:black.
Metadata: {'product_name': 'Bracketron Tour Window Hip Kicker', 'overall_rating': '4', 'product_category_tree': '"automotive, accessories & spare parts, car electronics & accessories, car mobile accessories, car mobile holders"', 'brand': 'bracketron'}
Dissimilarity score: 0.8901824951171875

 --------------------------------------------------
Content: Feature  in the box:mobile holder with adjustable tool. Feature  brand:fly. Feature  model:mini clip mobile holder. Feature  color:black.
Metadata: {'product_name': 'Fly Mini Clip Mobile Holder', 'overall_rating': '3.6', 'product_category_tree': '"automotive, accessories & spare parts, car electronics & accessories, car mobile accessories, car mobile holders"', 'brand': 'fly'}
Dissimilarity score: 0.9018765687942505

 --------------------------------------------------
Content: Feature  connectors:micro. Feat